In [324]:
import pandas as pd
import panel as pn
import param 
pn.extension()
import hvplot as hv
import hvplot.pandas
import holoviews as hv
from holoviews import opts# noqa

pn.extension()

In [325]:
data = pd.read_csv('a-Liste des adhérents_cleaned.csv')
data.head()

,Métropole,Date d'adhésion,Prénom,NOM,nb Adhérents,indiv,famille,Nom des membres famille,Adresse mail,Lieu de résidence,...,préparation et tenue de stand,élaboration de supports de communication,veille sur l'actualité du vélo,amélioration de la réglementation,atelier de réparation,cartographie collaborative,"autre, précisez.2",Unnamed: 43,Unnamed: 44,Vos attentes
0,x,21/03/2014,Manuel,LEGER,1.0,NaN,NaN,NaN,leger_manuel@yahoo.fr,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,x,21/03/2014,Emmanuelle,SAILLY,1.0,NaN,NaN,NaN,NaN,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,29/03/2014,Luc,MICHEL,1.0,NaN,NaN,NaN,mail supprimé à la demande de Luc Michel,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voies protégées. Vélos électriques à prix rais...
3,NaN,29/03/2014,Michèle,JOHN,1.0,NaN,NaN,NaN,michele.c.john@gmail.com,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,29/03/2014,Daniel,BRETAUDEAU,1.0,NaN,NaN,NaN,danielbretaudeau@gmail.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleanning du dataset

In [326]:
data["date_adhesion"] = pd.to_datetime(data["Date d'adhésion"],format='%d/%m/%Y')

In [327]:
data.loc[data['nb Adhérents'] > 1, "montant_adhesion"] = '4000'
data.loc[data['nb Adhérents'] > 1, "famille"] = 1
data.loc[data['nb Adhérents'] == 1, "montant_adhesion"] = '2000'
data.loc[data['nb Adhérents'] == 1, "indiv"] = 1

cols = ["famille", "indiv","nb Adhérents"]
data[cols] = data[cols].fillna(0).astype(int)
data.head()

,Métropole,Date d'adhésion,Prénom,NOM,nb Adhérents,indiv,famille,Nom des membres famille,Adresse mail,Lieu de résidence,...,veille sur l'actualité du vélo,amélioration de la réglementation,atelier de réparation,cartographie collaborative,"autre, précisez.2",Unnamed: 43,Unnamed: 44,Vos attentes,date_adhesion,montant_adhesion
0,x,21/03/2014,Manuel,LEGER,1,1,0,NaN,leger_manuel@yahoo.fr,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-21,2000
1,x,21/03/2014,Emmanuelle,SAILLY,1,1,0,NaN,NaN,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-21,2000
2,NaN,29/03/2014,Luc,MICHEL,1,1,0,NaN,mail supprimé à la demande de Luc Michel,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voies protégées. Vélos électriques à prix rais...,2014-03-29,2000
3,NaN,29/03/2014,Michèle,JOHN,1,1,0,NaN,michele.c.john@gmail.com,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-29,2000
4,NaN,29/03/2014,Daniel,BRETAUDEAU,1,1,0,NaN,danielbretaudeau@gmail.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-29,2000


# Création d'une classe d'objet statistique DAV héritant de param

### Création d'un slider pour filtrer les données sur la date d'adhésion

In [328]:
class StatDav(param.Parameterized):
    
    # WIDGETS ------------------------------------------------
    # slider date
    date_slider = param.Integer(data["date_adhesion"].min().year,  bounds=(data["date_adhesion"].min().year, data["date_adhesion"].max().year))
    

    # METHODES ------------------------------------------------
    # création de la metode qui renvoit le dataframe filtré
    @param.depends('date_slider')
    def select_date(self):
        return data.loc[data["date_adhesion"].dt.year == self.date_slider]

#stat = StatDav(name="Statistique des adhésions de DAV")
#pn.Row(stat.param, stat.select_date)

### Ajout d'une métode renvoyant le dataframe filtré à partir d'une date.

In [329]:
class StatDav(param.Parameterized):
    
    # WIDGETS ------------------------------------------------
    # slider date
    date_slider = param.Integer(data["date_adhesion"].min().year,  bounds=(data["date_adhesion"].min().year, data["date_adhesion"].max().year))
    

    # METHODES ------------------------------------------------
    # création de la metode qui renvoit le dataframe filtré
    @param.depends('date_slider')
    def select_date(self):
        return data.loc[data["date_adhesion"].dt.year == self.date_slider]
    
    @param.depends('date_slider')
    def getDfFromYear(self):
        filter = data["date_adhesion"].dt.year >= self.date_slider
        df_year = data[filter]
        return df_year[["NOM","Prénom","date_adhesion","famille","indiv","nb Adhérents","montant_adhesion"]]



### Ajout d'un widget et d'une métode renvoyant le dataframe filtré sur le type d'adhésion

In [330]:
class StatDav(param.Parameterized):    
  
   # WIDGETS ------------------------------------------------
    # slider date
    date_slider = param.Integer(data["date_adhesion"].min().year,  bounds=(data["date_adhesion"].min().year, data["date_adhesion"].max().year))
      
    # Radio bouton    
    type_adhesion = param.Selector(objects=['famille', 'indiv'])
    
    # METHODES ------------------------------------------------
    # création de la metode qui renvoit le dataframe filtré
    @param.depends('date_slider')
    def select_date(self):
        return data.loc[data["date_adhesion"].dt.year == self.date_slider]
    
    @param.depends('date_slider')
    def getDfFromYear(self):
        filter = data["date_adhesion"].dt.year >= self.date_slider
        df_year = data[filter]
        return df_year[["NOM","Prénom","date_adhesion","famille","indiv","nb Adhérents","montant_adhesion"]]

    @param.depends('date_slider','type_adhesion')
    def getDfByTypeAdhesion(self):
        df = getDfFromYear(self.date_slider)
        newDf= df.loc[(df[self.type_adhesion]==1)]
        return newDf

    

### Ajout d'un widget et d'une métode renvoyant le nombre d'adhésion correspondant aux différents choix des widgets précédents

In [450]:
class StatDav(param.Parameterized):    
  
   # WIDGETS ------------------------------------------------
    # slider date
    date_slider = param.Integer(data["date_adhesion"].min().year,  bounds=(data["date_adhesion"].min().year, data["date_adhesion"].max().year))
      
    # Radio bouton    
    type_adhesion = param.Selector(objects=['famille', 'indiv'])
    
    montant_adhesion_individuel = param.Integer(2000, bounds=(100,3000) )
    montant_adhesion_famille = param.Integer(4000, bounds=(100,5000))
    
    ratio_fammille_indiv = param.Number( default=0.3,  bounds=(0.1,1))

    
        
    # METHODES ------------------------------------------------
    # création de la metode qui renvoit le dataframe filtré
    @param.depends('date_slider')
    def select_date(self):
        return data.loc[data["date_adhesion"].dt.year == self.date_slider]
    
    @param.depends('date_slider')
    def getDfFromYear(self):
        filter = data["date_adhesion"].dt.year >= self.date_slider
        df_year = data[filter]
        return df_year[["NOM","Prénom","date_adhesion","famille","indiv","nb Adhérents","montant_adhesion"]]

    
    @param.depends('date_slider')
    def yearStat(self):
        df = self.getDfFromYear()
        statDict = {
            'adhesion_indiv' : len(df.loc[(df["indiv"]==1)]), 
            'adhesion_famille' : len(df.loc[(df["famille"]==1)]),
            'total' : len(df.loc[(df["indiv"]==1)]) +  len(df.loc[(df["famille"]==1)]),
            'adhérent' : int(df.loc[(df["famille"]==1)]["nb Adhérents"].sum()) + len(df.loc[(df["indiv"]==1)]),
            'ratio' :  round(len(df.loc[(df["famille"]==1)]) / (len(df.loc[(df["indiv"]==1)]) +  len(df.loc[(df["famille"]==1)])),2)
        }
        return statDict.get('total'), statDict.get('adhesion_indiv'), statDict.get('adhesion_famille'),  statDict.get('adhérent'), statDict.get('ratio')
   
    @param.depends('date_slider')
    def butifulValues(self):
        tuple = self.yearStat()
        tresor = self.getTresor()
        return pn.pane.Markdown("""
        # Depuis l'année %s """ %self.date_slider + """
        ## %s adhésions
        
        - %s adhésions individuelles 
        - %s adhésion famille 
        
        ## %s adhérents 
        
        il y a %s adhésions famille pour 1 adhésion individuelle 
        
        """ %tuple +
        """## %s Francs
        """ % tresor
                               
                               )
    
    @param.depends('date_slider','type_adhesion')
    def getDfByTypeAdhesion(self):
        df = self.getDfFromYear()
        newDf= df.loc[(df[self.type_adhesion]==1)]
        return newDf  
    
    @param.depends('date_slider','type_adhesion')
    def getStat(self):
        df = self.getDfByTypeAdhesion()
        self.simple_stat = len(df)
        return len(df)
    
    @param.depends('date_slider','montant_adhesion_individuel','montant_adhesion_famille')
    def getTresor(self):
        statDf = self.yearStat()
        montant_tresor = statDf[1] * self.montant_adhesion_individuel + statDf[2] * self.montant_adhesion_famille
        return montant_tresor
    
    @param.depends('date_slider')
    def bar(self):
        df =  self.getDfFromYear()
        famille = df.loc[(df["famille"]==1)]
        bar = famille.hvplot.hist(y='nb Adhérents').opts(title="Nombre d'adhérent par famille",ylabel="Nb adhésion famille", yticks=range(1,50),xticks=[1,2,3,4,5,6],padding=1)
        moyenne = famille["nb Adhérents"].mean()
        moyenne_line = hv.VLine(moyenne)
        layout = bar * moyenne_line
        layout.opts(opts.VLine(color='red', line_width=4))
        return print(type(famille))
    
    @param.depends('montant_adhesion_individuel','montant_adhesion_famille', 'ratio_fammille_indiv')
    def line(self):
        df = self.calculateDataset()
        #line = df.hvplot.line()
        #layout = line
        return print(type(df))
   
        
    @param.depends('montant_adhesion_famille', 'ratio_fammille_indiv','montant_adhesion_individuel')
    def calculateDataset(self):
        dfm = pd.DataFrame({'nb_adherents': range(1,200)})
        df =  self.getDfFromYear()
        famille = df.loc[(df["famille"]==1)]
        moyenne = famille["nb Adhérents"].mean()
        dfm["nb_adhesion"] =  dfm['nb_adherents'] * moyenne    
        dfm["nb_adhesion_calc"] = dfm["nb_adhesion"] * self.ratio_fammille_indiv  
        dfm["tresor_calc_ratio_famille"] = round(((dfm['nb_adherents'] * self.ratio_fammille_indiv )* self.montant_adhesion_famille)+ (dfm['nb_adherents'] *(1- self.ratio_fammille_indiv)* self.montant_adhesion_individuel ))
        return pn.panel(dfm,width=1000)



In [451]:
stat = StatDav(name="Statistique des adhésions de DAV")

app =  pn.Column(pn.Column(pn.Param(stat.param['date_slider']),pn.Row(stat.butifulValues, stat.bar)))
app

<class 'pandas.core.frame.DataFrame'>


Column
    [0] Column
        [0] Param(StatDav, parameters=['date_slider'], show_name=False)
        [1] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)

In [452]:
app =  pn.Column(pn.Row(pn.Column(pn.Param(stat.param, widgets={'type_adhesion':  pn.widgets.RadioButtonGroup}))),stat.line)
app

<class 'panel.pane.markup.DataFrame'>


Column
    [0] Row
        [0] Column
            [0] Param(StatDav, widgets={'type_adhesion': <...})
    [1] ParamMethod(method)

In [ ]:
def formatter(value):
    return str(value) 

In [ ]:
nbAdherent_line = hv.VLine(nbAdherent).opts(alpha=0.5,color='grey', line_width=2,line_dash='dashed')

famille_points = hv.Points((adhesion_famille,adhesion_famille*4000)).opts( size=8)
indiv_point = hv.Points((adhesion_indiv,adhesion_indiv*2000)).opts(color='green', size=8)
famille_adherents_points = hv.Points((nbAdherent_famille,adhesion_famille*4000)).opts(size=8)

total_point = hv.Points((nb_adhesion,montant_tresor)).opts(color='grey', size=8)

famille_Hline = hv.HLine(adhesion_famille*4000).opts(alpha=0.3,color='blue',line_width=1)


layout = (dfm.hvplot.line(x="nb_adherents", y="tresor_4000") *
 dfm.hvplot.line(x="nb_adherents", y="tresor_3000") * 
 dfm.hvplot.line(x="nb_adherents", y="tresor_2000") * 
 dfm.hvplot.line(x="nb_adherents", y="tresor_calc_ratio_famille") * 

nbAdherent_line *
          famille_points *
          indiv_point * 
          total_point* 
         famille_Hline).opts(xlabel="Nb adhésions", ylabel="Trésor",yformatter=formatter,ylim=(0,500000),xlim=(0,150),show_grid=True)
layout

In [ ]:

print(row["nb_adhesion"])


In [ ]:

nb_adherent_r100_line = dfm.hvplot.line(x="nb_adherents", y="nb_adhesion")
dynamique_curve = dfm.hvplot.line(x="nb_adherents", y="nb_adhesion_calc")
nb_adherent_r1_line = dfm.hvplot.line(x="nb_adherents", y="nb_adherents")


layout_ratio = (nb_adherent_r100_line * dynamique_curve *nb_adherent_r1_line
).opts(xlabel="Nb adhérents", ylabel="Nb adhésions",yformatter=formatter,ylim=(0,500),xlim=(0,150),show_grid=True)

layout_ratio